# Chemical conc. runs

We use `blpd` to model canopy emissions (e.g., floral volatiles). Lagrangian particles have in-particle concentrations of chemical species, which can be spatially aggregated to form concentration maps (or at least relative to release).

In [ ]:
import matplotlib.pyplot as plt

import blpd
from blpd import plot

## Run LPD

For chemical concentrations, we use the (default) continuous-release run type.

In [ ]:
# Start from default case
m = blpd.model()

# Change some params
new_p = {
    "source_positions": [(-200, -200), (500, 0), (1000, 200)],
    # ^ similar to the ones Pratt used in their Fig 4.2
    "t_tot": 1 * 60.0,  # 1 min; change to 10 min to make the plots more exciting
    "continuous_release": True,  # this is a default
    "dNp_per_dt_per_source": 4,
}
m.update_p(**new_p)

# Run
m.run()

## Calculate chemistry

Chemistry is calculated offline, after the LPD model integration. Chemistry routines take the model output dataset (`xr.Dataset`) as input.

In [ ]:
ds0 = m.to_xarray()
ds0

In [ ]:
ds = blpd.chem.calc_relative_levels_fixed_oxidants(ds0)
ds

## Plots

`'apinene'` ([α-pinene](https://en.wikipedia.org/wiki/Alpha-Pinene)) is the default species plotted if `spc` is not specified.

In the `conc_2d` plot, particles are binned in two dimensions only ($x$ and $y$ by default), so the result is akin to something like total column ozone.

In [ ]:
plot.conc_2d(ds)

Below we demonstrate the impact of some `conc_2d` settings. The choice of `bins` is important.

In [ ]:
kwargss = [
    dict(),  # default
    dict(bins=(100, 50)),
    dict(bins="auto"),
    dict(bins="auto", log_cnorm=True),
    dict(plot_type="contourf"),
    dict(plot_type="contourf", bins=(100, 50)),
    dict(plot_type="contourf", bins="auto"),
    dict(plot_type="contourf", bins="auto", log_cnorm=True),
]

fig, axs = plt.subplots(4, 2, sharex=True, sharey=True, figsize=(9, 12))

for ax, kwargs in zip(axs.flat, kwargss):
    plot.conc_2d(ds, **kwargs, ax=ax)

We can compare the above reactive species conc. plots to one of the non-reactice particles.

In [ ]:
plot.final_pos_hist2d(ds, bins="auto", log_cnorm=True)

We can plot other chemical species if we want.

In [ ]:
fig, axs = plt.subplots(3, 2, sharex=True, sharey=True, figsize=(9, 9))

for spc, ax in zip(ds.spc.values[:6], axs.flat):
    plot.conc_2d(ds, spc, plot_type="pcolor", bins="auto", log_cnorm=True, vmin=1.0, ax=ax)
    ax.text(
        0.02,
        0.98,
        f"mean: {ds.f_r.sel(spc=spc).mean().values:.4g}",
        ha="left",
        va="top",
        transform=ax.transAxes,
    )

And "line" concentrations.

In [ ]:
plot.conc_xline(ds, spc="all")